In [5]:
import xml.etree.ElementTree as ET
import os

In [6]:
# 1. Read roads' connection relationships from Sumo's road network .xml file.

# Define data path.
path = os.path.abspath('./../SUMO_Simulation')
xml_data = os.path.join(path, 'test.net.xml')
print('network path is:', xml_data)

# Extract .xml file.
tree = ET.parse(xml_data)
root = tree.getroot()
# Initialize dictionary.
from_to_dict = {}   # Map road ID forward to its connected road IDs with turning direction
to_from_dict = {}   # Map road ID backward to which road IDs can drive to the current one with turning direction

for conn in root.findall('connection'):
    from_edge = conn.get('from')
    to_edge = conn.get('to')
    dir = conn.get('dir')

    # Ignore 'U-turn'
    if dir == 't':
        continue

    # Construct forward dictionary
    if from_edge not in from_to_dict:
        from_to_dict[from_edge] = []
    if (to_edge, dir) not in from_to_dict[from_edge]:
        from_to_dict[from_edge].append((to_edge, dir))

    # Construct backward dictionary
    if to_edge not in to_from_dict:
        to_from_dict[to_edge] = []
    if (from_edge, dir) not in to_from_dict[to_edge]:
        to_from_dict[to_edge].append((from_edge, dir))

# Print length of each dictionary
print("From-To Dictionary:", len(from_to_dict))
print("To-From Dictionary:", len(to_from_dict))

network path is: /Users/xuzizhuo/Desktop/Main Folder/My_works/routeSys_Github/Traffic_Simulation_Data_Generation_for_Model_Training/SUMO_Simulation/test.net.xml
From-To Dictionary: 29381
To-From Dictionary: 29362


In [7]:
# 2. Read selected road IDs
with open('classfied_edge_ids.txt', 'r') as file:
    edge_ids = [line.strip() for line in file]

In [8]:
# 3. Generate routes based on selected road IDs in each category.

# Define the length of roads in each route
length = 3

# For the target road, select three connected roads forward and backward to generate a route
def generate_routes(edge_id, from_to_dict, to_from_dict, n=length, visited_global=set()):
    # Initialization
    routes = []
    count = 0
    repeat_count = 0
    terminated_routes = []

    if edge_id in from_to_dict:
        # Find forward edges
        for to, dir_main in from_to_dict[edge_id]:
            count += 1                
            prev_edges = []
            next_edges = [to]
            visited_edges = {edge_id, to}
            current = edge_id
            
            # Find 'from' edge IDs (left direction, backward)
            for _ in range(n):
                if current in to_from_dict:
                    found = False
                    for from_edge, dir in to_from_dict[current]:
                        if from_edge not in visited_edges:
                            current = from_edge
                            prev_edges.append(current)
                            visited_edges.add(current)
                            found = True
                            break
                    if not found: # Terminate generation procedure
                        terminated_routes.append((prev_edges[::-1] + [edge_id] + next_edges, "terminated"))
                        break
                else:
                    # print(f"Warning. edge ID {current} does not contain in dictionary to_from_dict as key. Edge ID may not have 'to' relationship in network data.")
                    break

            current = to
            # Find 'to' edge IDs (right direction, forward)
            for _ in range(n - 1):
                if current in from_to_dict:
                    found = False
                    for to_edge, dir in from_to_dict[current]:
                        if to_edge not in visited_edges:
                            current = to_edge
                            next_edges.append(current)
                            visited_edges.add(current)
                            found = True
                            break
                    if not found: # Terminate generation procedure
                        terminated_routes.append((prev_edges[::-1] + [edge_id] + next_edges, "terminated"))
                        break
                else:
                    # print(f"Warning. edge ID {current} does not contain in dictionary from_to_dict as key. Edge ID may not have 'from' relationship in network data.")
                    break

            # Combine found road IDs to generate route
            route = prev_edges[::-1] + [edge_id] + next_edges
            
            # Check if edge IDs already contained in other generated routes, if repeat, do not use this one
            # Make sure all routes do not share same road IDs
            if not set(route) & visited_global:
                routes.append((route, dir_main))
                visited_global.update(route)
            else:
                repeat_count += 1
    else:
        # print(f"Warning. Edge ID {edge_id} is not contained in dictionary from_to_dict as key. Edge ID may not have 'from' relationship in network data.")
        pass
    
    return routes, count, repeat_count, terminated_routes


all_routes = {}
terminated_routes = []
visited_global = set()
only_to_edges = []
all_route_num = 0
counts = 0
repeat_count = 0
repeat_count_sum = 0

# Routes generation
for edge_id in edge_ids:
    if edge_id in from_to_dict:
        routes, count, repeat_count, terminated = generate_routes(edge_id, from_to_dict, to_from_dict, visited_global=visited_global)        
        # Dictionary store generated routes, map target road ID to the generated route
        all_routes[edge_id] = routes
        terminated_routes.extend(terminated)
        # All route nubmer incude same target edge with different directions
        all_route_num += count
        counts += len(routes)
        repeat_count_sum += repeat_count
    else:
        only_to_edges.append(edge_id)

print("Number of target routes:", len(all_routes))
print("Edges only in 'to' but not in 'all_routes':", only_to_edges)

print(f"All required route number: {all_route_num}")
print(f"Satisfied route number: {counts}")
# Include in satisfied routes but not intact
print(f"Terminated route number: {len(terminated_routes)}")
# Unsatisfied routes and the sum of satisfied and unsatisfied should equal to all required route number
print(f"Unsatisfied route number {repeat_count_sum}")


Number of target routes: 471
Edges only in 'to' but not in 'all_routes': ['320888292', '46177152', '198924631', '5681925', '658498546', '46179217', '-5671365#0', '25464382', '46492008', '9702602', '521358284', '546364858', '285331372#1', '5670010#1']
All required route number: 538
Satisfied route number: 350
Terminated route number: 0
Unsatisfied route number 188


In [9]:
# 4. Check if all routes share the same road IDs

def check_duplicates(routes):
    duplicate_routes = {}
    
    for route, direction in routes:
        # Check duplicates by the comparing the length of set and list
        if len(set(route)) != len(route):
            duplicate_routes[(tuple(route), direction)] = True
        else:
            duplicate_routes[(tuple(route), direction)] = False
            
    return duplicate_routes


# Check all routes
all_routes_list = [route for routes in all_routes.values() for route in routes]
duplicates_found = check_duplicates(all_routes_list)
# Print results
total_routes = len(all_routes_list)
duplicate_routes_count = sum(1 for has_duplicate in duplicates_found.values() if has_duplicate)
print("Total number of routes:", total_routes)
print("Number of routes with duplicates:", duplicate_routes_count)

Total number of routes: 350
Number of routes with duplicates: 0


In [10]:
# 5. Check generated routes are connected

# For single route, check if all road IDs are connected
def is_route_connected(route, from_to_dict, to_from_dict):
    for i in range(len(route) - 1):
        from_edge = route[i]
        to_edge = route[i+1]
        if not (from_edge in from_to_dict and any(to == to_edge for to, _ in from_to_dict[from_edge])):
            print("Error. Route is not connected.")
            return False
    return True

# Check connection for all routes
connected_routes = {}
for edge_id, routes in all_routes.items():
    for route, dir in routes:
        if is_route_connected(route, from_to_dict, to_from_dict):
            if edge_id not in connected_routes:
                connected_routes[edge_id] = []
            connected_routes[edge_id].append((route, dir))

# Print result
print("Connected Routes:", len(connected_routes))


Connected Routes: 350


In [11]:
# 6. Capture roads' static information and connected relationship from Sumo's network file.

# Capture roads' static information based on each lane.
# E.g. each road contains at least one lane and we average speed limit and length of all lanes for each road.
def parse_net_file(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    edge_dict = {}
    
    for edge in root.findall('edge'):
        edge_id = edge.get('id')
        lanes = edge.findall('lane')
        lane_info = []
        
        for lane in lanes:
            lane_id = lane.get('id')
            length = float(lane.get('length'))
            speed = float(lane.get('speed'))
            lane_info.append((lane_id, length, speed))

        edge_dict[edge_id] = lane_info
    
    return edge_dict

# Capture the connected relationship.
def parse_connections(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    connection_dict = {}
    
    for conn in root.findall('connection'):
        from_edge = conn.get('from')
        dir = conn.get('dir')
        if (dir == 't'):
            continue
            
        from_lane = conn.get('fromLane')
        key = (from_edge, dir)
        
        if key not in connection_dict:
            connection_dict[key] = {
                'lane_ids': [],
                'total_length': 0,
                'total_speed': 0
            }
        
        connection_dict[key]['lane_ids'].append(from_lane)
    
    return connection_dict


# Combine these two dictionary to generate a new dictionary. 
def combine_edge_and_connection(edge_dict, connection_dict):
    result_dict = {}
    
    for key, conn_info in connection_dict.items():
        from_edge, dir = key
        lane_ids = conn_info['lane_ids']
        
        if from_edge in edge_dict:
            lanes = edge_dict[from_edge]
            
            total_length = 0
            total_speed = 0
            lane_count = 0
            
            for lane_id, length, speed in lanes:
                if lane_id.split('_')[1] in lane_ids:
                    total_length += length
                    total_speed += speed
                    lane_count += 1
            
            result_dict[key] = {
                'lane_count': lane_count,
                'avg_length': total_length / lane_count if lane_count > 0 else 0,
                'avg_speed': total_speed / lane_count if lane_count > 0 else 0
            }
    
    return result_dict

# Define path
path = os.path.abspath('./../SUMO_Simulation')
net_file_path = os.path.join(path, 'test.net.xml')

edge_dict = parse_net_file(net_file_path)
connection_dict = parse_connections(net_file_path)
result = combine_edge_and_connection(edge_dict, connection_dict)

'''
# Print result
for key, value in result.items():
    print(f"Edge ID: {key[0]}, Direction: {key[1]}")
    print(f"Lane Count: {value['lane_count']}, Avg Length: {value['avg_length']}, Avg Speed: {value['avg_speed']}")
'''

'\n# Print result\nfor key, value in result.items():\n    print(f"Edge ID: {key[0]}, Direction: {key[1]}")\n    print(f"Lane Count: {value[\'lane_count\']}, Avg Length: {value[\'avg_length\']}, Avg Speed: {value[\'avg_speed\']}")\n'

In [12]:
# 7. Export route data as .xml file which can be loaded in Sumo

def get_edge_info(edge_id, direction, result_dict):
    key = (edge_id, direction)
    if key in result_dict:
        return result_dict[key]
    else:
        print(f"Error. Cannot find {edge_id} and {direction} in map")
        return None
    
def prettify(elem, level=0):
    i = "\n" + level*"  "
    if len(elem):
        if not elem.text or not elem.text.strip():
            elem.text = i + "  "
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
        for elem in elem:
            prettify(elem, level+1)
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = i

def write_sumo_xml(routes):
    root = ET.Element('routes')
    root.set("xmlns:xsi", "http://www.w3.org/2001/XMLSchema-instance")
    root.set("xsi:noNamespaceSchemaLocation", "http://sumo.dlr.de/xsd/routes_file.xsd")

    vehicle_id = 0
    cycles_per_flow = 5
    vehicles = []
    base_depart = 0.0  # 初始化base_depart在循环之外

    for edge_id, route_dirs in routes.items():        
        for route, dir in route_dirs:
            # Define the number of routes can departure at the same time
            max_route = 20

            info = get_edge_info(edge_id, dir, result)
            lane_count = info['lane_count']
            avg_length = info['avg_length']
            avg_speed = info['avg_speed']
            
            if (lane_count is None or avg_length is None or avg_speed is None):
                print(f"Missing data for Edge ID: {edge_id}, Direction: {direction}")
                capacity = max_route
            else:
                capacity = int(lane_count * avg_length / 5)
                # print(f"lane count: {lane_count}")
                # print(f"ave length: {avg_length}")
                # print(capacity)
            
            if (max_route >= capacity):
                max_route = capacity
                
            edges = " ".join(route)
            for multiplier in range(1, max_route + 1):  # Routes depart at the same time from 1 to 20
                for repeat in range(cycles_per_flow):   # Repeat 10 times
                    for i in range(multiplier):
                        depart_time = base_depart + 0.1 * i
                        vehicles.append((depart_time, vehicle_id, edges))
                        vehicle_id += 1
                    base_depart += 30.0  # Departure time add 30 in case of congestion
    
    print("simulation time is: :", base_depart)
    
    vehicles.sort()
    for depart_time, vehicle_id, edges in vehicles:
        vehicle = ET.SubElement(root, 'vehicle', id=str(vehicle_id), depart=f"{depart_time:.2f}")
        ET.SubElement(vehicle, 'route', edges=edges)

    prettify(root)
    tree = ET.ElementTree(root)
    
    # Define path
    path = os.path.abspath('./../SUMO_Simulation') 
    data_path = os.path.join(path,'classified_edges.rou.xml') 
    print('Data path is:', data_path)

    tree.write(data_path, encoding='utf-8', xml_declaration=True)

    return vehicle_id

total_vehicles = write_sumo_xml(all_routes)
print(f"Total vehicles generated: {total_vehicles}")


simulation time is: : 677700.0
Data path is: /Users/xuzizhuo/Desktop/Main Folder/My_works/routeSys_Github/Traffic_Simulation_Data_Generation_for_Model_Training/SUMO_Simulation/classified_edges.rou.xml
Total vehicles generated: 214919
